# Bomen

Bomen vormen belangrijke structuur in informatica:

* filesysteem (mappen en bestanden)
* zoekbomen (o.a. B-trees in databases)
* ontleedbomen, expressiebomen
* HTML DOM (tussen HTML en rendering)
* ...


Ook buiten informatica vind je bomen:

* hiërarchische structuur
    * organisaties
    * boek (inhoudsopgave)
* biologie: takkenstructuur, wortelstructuur, longen, ...
* rivieren

Voorbeeld: grafische figuren in Elm

* verschillende soorten figuren: lijnstuk, rechthoek, cirkel, ...
* groep van figuren (handig voor herhaling e.d.)
* verplaatsen van figuren (translate; rotate; scale)

Gebruik van *Custom Types* (ook wel *union types*)

Binaire boom als Custom type:

```elm
type Tree 
  = Node String Tree Tree  -- 1e alternatief
  | Nil                    -- 2e alternatief
```

List als Custom type:

``` Elm
type List elt
  = Cons elt (List elt)    -- 1e alternatief
  | Empty                  -- 2e alternatief
```

Waarin `Cons` de rol heeft van `::` - `Cons a lst = a :: lst`

Enkele voorbeelden van Tree-waarden:

```elm
tree45 = Node "*" (Node "4" Nil Nil) (Node "5" Nil Nil)
tree345 = Node "+" (Node "3" Nil Nil) tree45 

a = Node "jet" (Node "aap" Nil Nil) (Node "mies" Nil Nil)
b = Node "wim" (Node "vuur" Nil Nil) (Node "zus" Nil Nil)
c = Node "noot" a b
```

Verschillende manieren om boom in string om te zetten:
prefix, infix, postfix

```elm
prefix tree =
  case tree of
    Node str left right -> 
      str ++ " " ++ (prefix left) ++ " " ++(prefix right)
      
    Nil -> 
      ""
```

In [ ]:
import Html exposing (..)
import String exposing (..)

type Tree 
  = Node String Tree Tree  -- 1e alternatief
  | Nil                    -- 2e alternatief
  
tree45 = Node "*" (Node "4" Nil Nil) (Node "5" Nil Nil)
tree345 = Node "+" (Node "3" Nil Nil) tree45 

prefix tree =
  case tree of
    Node str left right -> 
      str ++ " " ++ (prefix left) ++ " " ++(prefix right)     
    Nil -> 
      ""

main =
  text (prefix tree345)
-- compile-code

## Binaire zoekboom

* waarden links <= waarde wortel <= waarden rechts
* gebalanceerde binaire zoekboom: zoeken O(N log N).

![binaire zoekboom](bin-zoekboom.png)

In [ ]:
import Html exposing (..)
import String exposing (..)

type Tree 
  = Node String Tree Tree  -- 1e alternatief
  | Nil                    -- 2e alternatief
  
a = Node "jet" (Node "aap" Nil Nil) (Node "mies" Nil Nil)
b = Node "wim" (Node "vuur" Nil Nil) (Node "zus" Nil Nil)
c = Node "noot" a b

infix tree =
  case tree of
    Node str left right -> 
     (infix left) ++ " " ++ str ++ " " ++(infix right)     
    Nil -> 
      ""

main =
  text (infix c)
-- compile-code

**Opdrachten**

* pas dit programma aan om de boom in *postfix* af te drukken: eerste de kind-bomen, dan de wortel.
* gebruik de boom voor de expressie `3 + 4 * 5`, en druk deze in postfix af
* dat geeft de rekenvolgorde van een HP rekenmachine (stack machine):
    * push 3; push 4; push 5; mul; add
* definieer een boom voor de expressie `(3 + 4) * 5`
* druk deze boom in postfix af
* vergelijk de postfix-reeks met de vorige

## N-aire boom

Voorbeeld: inhoudsopgave van een boek

Andere voorbeelden: Html, Svg bomen ("DOM" in browser)

In [ ]:
import Html exposing (..)
import String exposing (..)

type Tree a
  = Node a (List (Tree a))

type alias Toc = Tree String

In [ ]:
printToc : Toc -> List (Html msg)
printToc toc  =
  printSubtocs [toc] ""

printSubtocs : List Toc -> String -> List (Html msg)
printSubtocs tocs pref =
  case tocs of
    (Node x s) :: xs ->
      let 
        head = Html.div [] [Html.text (pref ++ " " ++ x)]
        subs = printSubtocs s (pref ++ "...")
        next = printSubtocs xs pref
      in
        head :: subs ++ next
    
    [] ->
      []

In [ ]:
toc1 = Node "Titel" [hoofdstuk1, hoofdstuk2, hoofdstuk3]
hoofdstuk1 = Node "Hoofdstuk 1" [sectie11, sectie12]
sectie11 = Node "Sectie1-1" []
sectie12 = Node "Sectie1-2" []
hoofdstuk2 = Node "Hoofdstuk 2" [sectie21, sectie22, sectie23]
sectie21 = Node "Sectie2-1" []
sectie22 = Node "Sectie2-2" []
sectie23 = Node "Sectie2-3" []
hoofdstuk3 = Node "Hoofdstuk 3" [sectie31, sectie32]
sectie31 = Node "Sectie2-1" []
sectie32 = Node "Sectie2-2" []


main = div [] (printToc toc1)

In [ ]:
-- compile-code

**Opdracht**

* Pas dit programma aan: voeg een 4e sectie toe aan hoofdstuk 2
* is het uitvoer-formaat: infix, prefix, postfix?
* waar kom je een dergelijke layout van een boom nog meer tegen?

## SVG

Voorbeeld van een SVG-figuur als boomstructuur: `circle` en `rect` zijn sub-bomen van `svg`.

In [ ]:
import Svg exposing (svg, circle, rect)
import Svg.Attributes exposing (..)

main =
  svg
    [ width "200", height "200", viewBox "0 0 200 200" ]
    [ circle [ cx "50", cy "50", r "50", fill "green" ] []
    , rect [x "40", y "90", width "20", height "80", fill "burlywood" ] []
    ]

-- compile-code

## Afdrukken van een boom

 

In [ ]:
import Svg exposing (..)
import Svg.Attributes exposing (..)
import String exposing (fromInt, length)

type Tree 
  = Node String Tree Tree
  | Nil

In [ ]:
translate : (Int, Int) -> List (Svg msg) -> Svg msg
translate (x, y) lst =
  Svg.g
    [transform ("translate(" ++ fromInt(x) ++ "," ++ fromInt(y) ++ ")")]
    lst

unitwidth = 30
unitheight = 50

In [ ]:
twidth : Tree -> (Int, Int)
twidth t =
  case t of        
    Node str left right ->
      let
        (lw, _) = twidth left
        (rw, _) = twidth right
      in
        (lw + unitwidth + rw, lw + (unitwidth // 2))

    Nil ->
      (0, 0)
      
swidth str =
  length str * 4

In [ ]:
drawLine : (Int, Int) -> (Int, Int) -> Svg msg
drawLine (xa, ya) (xb, yb) =
  Svg.line
    [ x1 (fromInt xa)
    , y1 (fromInt ya)
    , x2 (fromInt xb)
    , y2 (fromInt yb)
    , stroke "black"
    , strokeWidth "1"    
    ]
    []

In [ ]:
vspace = unitheight - 20

drawTree : Tree -> Svg msg
drawTree t =
  case t of
    Node str Nil Nil ->
      translate (unitwidth // 2 - swidth str, 0) [Svg.text_ [] [text str]]
      
    Node str Nil right ->
      let
        (rw, rm) = twidth right
        rtree = translate (unitwidth, unitheight) [drawTree right]
        node = translate (unitwidth // 2 - swidth str, 0) [Svg.text_ [] [text str]]
        rightedge = drawLine (unitwidth // 2, 5) (unitwidth + rm, vspace)
      in
        Svg.g [] [node, rtree, rightedge]

    Node str left Nil ->
      let
        (lw, lm) = twidth left
        ltree = translate (0, unitheight) [drawTree left]
        node = translate (lw + unitwidth // 2 - swidth str, 0) [Svg.text_ [] [text str]]
        leftedge = drawLine (lw + unitwidth // 2, 5) (lm, vspace)
      in
        Svg.g [] [ltree, node, leftedge]     
        
    Node str left right ->
      let
        (lw, lm) = twidth left
        (rw, rm) = twidth right
        ltree = translate (0, unitheight) [drawTree left]
        rtree = translate (lw + unitwidth, unitheight) [drawTree right]
        node = translate (lw + unitwidth // 2 - swidth str, 0) [Svg.text_ [] [text str]]
        leftedge = drawLine (lw + unitwidth // 2, 5) (lm, vspace)
        rightedge = drawLine (lw + unitwidth // 2, 5) (lw + unitwidth + rm, vspace)
      in
        Svg.g [] [ltree, node, rtree, leftedge, rightedge]
    Nil ->
      Svg.text_ [] [text "."]
      

In [ ]:
tree345 = Node "+" (Node "3" Nil Nil)  (Node "*" (Node "4" Nil Nil) (Node "5" Nil Nil) )

a = Node "jet" (Node "aap" Nil Nil) (Node "mies" Nil Nil)
b = Node "wim" (Node "vuur" Nil Nil) (Node "zus" Nil Nil)
c = Node "noot" a b

main =
  svg
    [ width "600", height "300", viewBox "0 0 600 300" ]
    [ translate (20, 50) [drawTree c]
    , translate (400, 50) [drawTree tree345]
    ]
    
-- compile-code

**Opdrachten**

* Teken de expressieboom voor `(3 + 4) * 5`.

In [ ]:
import Svg exposing (..)
import Svg.Attributes exposing (..)
import String exposing (fromInt, length)

type Tree 
  = Node String Tree Tree
  | Nil

translate : (Int, Int) -> List (Svg msg) -> Svg msg
translate (x, y) lst =
  Svg.g
    [transform ("translate(" ++ fromInt(x) ++ "," ++ fromInt(y) ++ ")")]
    lst

unitwidth = 30
unitheight = 50

twidth : Tree -> (Int, Int)
twidth t =
  case t of        
    Node str left right ->
      let
        (lw, _) = twidth left
        (rw, _) = twidth right
      in
        (lw + unitwidth + rw, lw + (unitwidth // 2))

    Nil ->
      (0, 0)
      
swidth str =
  length str * 4

drawLine : (Int, Int) -> (Int, Int) -> Svg msg
drawLine (xa, ya) (xb, yb) =
  Svg.line
    [ x1 (fromInt xa)
    , y1 (fromInt ya)
    , x2 (fromInt xb)
    , y2 (fromInt yb)
    , stroke "black"
    , strokeWidth "1"    
    ]
    []

vspace = unitheight - 20

drawTree : Tree -> Svg msg
drawTree t =
  case t of
    Node str Nil Nil ->
      translate (unitwidth // 2 - swidth str, 0) [Svg.text_ [] [text str]]
      
    Node str Nil right ->
      let
        (rw, rm) = twidth right
        rtree = translate (unitwidth, unitheight) [drawTree right]
        node = translate (unitwidth // 2 - swidth str, 0) [Svg.text_ [] [text str]]
        rightedge = drawLine (unitwidth // 2, 5) (unitwidth + rm, vspace)
      in
        Svg.g [] [node, rtree, rightedge]

    Node str left Nil ->
      let
        (lw, lm) = twidth left
        ltree = translate (0, unitheight) [drawTree left]
        node = translate (lw + unitwidth // 2 - swidth str, 0) [Svg.text_ [] [text str]]
        leftedge = drawLine (lw + unitwidth // 2, 5) (lm, vspace)
      in
        Svg.g [] [ltree, node, leftedge]     
        
    Node str left right ->
      let
        (lw, lm) = twidth left
        (rw, rm) = twidth right
        ltree = translate (0, unitheight) [drawTree left]
        rtree = translate (lw + unitwidth, unitheight) [drawTree right]
        node = translate (lw + unitwidth // 2 - swidth str, 0) [Svg.text_ [] [text str]]
        leftedge = drawLine (lw + unitwidth // 2, 5) (lm, vspace)
        rightedge = drawLine (lw + unitwidth // 2, 5) (lw + unitwidth + rm, vspace)
      in
        Svg.g [] [ltree, node, rtree, leftedge, rightedge]
    Nil ->
      Svg.text_ [] [text "."]
      

drie = Node "3" Nil Nil
vier = Node "4" Nil Nil
vijf = Node "5" Nil Nil
tree345 = Node "+" drie  (Node "*" vier vijf )

a = Node "jet" (Node "aap" Nil Nil) (Node "mies" Nil Nil)
b = Node "wim" (Node "vuur" Nil Nil) (Node "zus" Nil Nil)
c = Node "noot" a b

main =
  svg
    [ width "600"
    , height "300"
    , viewBox "0 0 600 300"
    ]
    [ translate (20, 50) [drawTree c]
    , translate (400, 50) [drawTree tree345]
    ]
    
-- compile-code